In [131]:
import pandas as pd
import numpy as np

import requests
import json
from bs4 import BeautifulSoup

import dill
import re
import time

Scrape the list of wines.

In [132]:
page_list = [range(0,52), range(0,22), range(0,2)]
url_list = ['http://www.wine.com/v6/Red-Wine/wine/list.aspx?N=7155+124&pagelength=100&Nao={}',
            'http://www.wine.com/v6/White-Wine/wine/list.aspx?N=7155+125&pagelength=100&Nao={}',
            'http://www.wine.com/v6/Rose-Wine/wine/list.aspx?N=7155+126&pagelength=100&Nao={}']
color_list = ['red', 'white', 'rose']

In [134]:
wine_urls = list()

for page_range, url_base, color in zip(page_list, url_list, color_list):
    print url_base
    for pg in page_range:
        url_no = 1+100*pg

        # Get the HTML
        req = requests.get(url_base.format(url_no))
        soup = BeautifulSoup(req.text, 'lxml')

        # Get the item list 
        item_list = soup.find(attrs={'class':'productList'}).find_all(attrs={'class':'verticalListItem'})
        item_list = [x.find('a',href=True).get('href') for x in item_list]
        item_list = [(color, 'http://www.wine.com'+x) for x in item_list]
        wine_urls.extend(item_list)

http://www.wine.com/v6/Red-Wine/wine/list.aspx?N=7155+124&pagelength=100&Nao={}
http://www.wine.com/v6/White-Wine/wine/list.aspx?N=7155+125&pagelength=100&Nao={}
http://www.wine.com/v6/Rose-Wine/wine/list.aspx?N=7155+126&pagelength=100&Nao={}


In [135]:
with open('../pkl/00_wine_dot_com_url_list.pkl','w') as fh:
    dill.dump(wine_urls, fh)

In [136]:
len(wine_urls)

7600

Scrape each of the web pages.

In [139]:
proxy1 = {'https': 'socks5://127.0.0.1:8081'}
proxy2 = {'https': 'socks5://127.0.0.1:8082'}
proxy3 = {'https': 'socks5://127.0.0.1:8083'}
proxy4 = {'https': 'socks5://127.0.0.1:8084'}
proxy_list = [proxy1, proxy2, proxy3, proxy4]

In [ ]:
for url in enumerate(wine_urls):
    
    if (url[0] % 100) == 0:
        print url[0]
        
        if url[0] != 0:
            wine_df = pd.concat(wine_df_list, axis=0).reset_index(drop=True)
            wine_df.to_pickle('../pkl/00_wine_dot_com_data_{}.pkl'.format(str(url[0])))
        
        wine_df_list = list()
    
    
    full_url = url[1][1]
    color = url[1][0]
    proxy = np.random.choice(proxy_list)
    html = requests.get(full_url, proxies=proxy)
    soup = BeautifulSoup(html.text, 'lxml')
    wine_text = soup.find(attrs={'class': 'productAbstract'})
    time.sleep(1.5)
    
    # wine
    try:
        wine = wine_text.find('h1').text.strip()
    except:
        wine = ''

    try:
        year = re.search(r"""((?:20|19)[0-9]{2})""", wine).group(1)
        wine = wine.replace(year, '').strip()
    except:
        year = ''
        
    # kind, region
    try:
        kind_loc_match = re.search(r"""(.+) from (.+)""", wine_text.find('h2').text.strip())
    except:
        kind = ''
        region = ''
    else:
        try:
            kind = kind_loc_match.group(1)
        except:
            kind = ''

        try:
            region = kind_loc_match.group(2)
        except:
            region = ''
            
    # review
    try:
        review = soup.find(attrs={'class':'tabContent aboutTheWine active'}).find(attrs={'itemprop':'description'}).text
    except:
        review = ''
        
    # winery
    try:
        winery = soup.find(attrs={'class':'tabContent theWinery'}).find('h3').text.strip()
    except:
        winery = ''
        
    # ratings
    try:
        ratings_list = [x.text.strip() 
                        for x in 
                        wine_text.find_all(attrs={'class': 'wineRatings'})]

        ratings_list = [re.findall(r"""((?:20|19)[0-9]{2}|[A-Z]{2}[0-9]{2})""", x) 
                        for x in ratings_list]

        recent = [float(re.search(r"""[0-9]+""", x).group(0)) for x in ratings_list[0]]
        if len(recent) >= 1:
            rating = np.array(recent).mean()
        else:
            rating = np.NaN
    except:
        rating = np.NaN
        
        
    df = pd.DataFrame({'wine':wine, 'year':year, 'kind':kind,
                       'region':region, 'review':review,
                       'winery':winery, 'rating':rating,
                       'color':color, 'url':full_url}, index=pd.Index([0]))
    
    wine_df_list.append(df)
    
    
wine_df = pd.concat(wine_df_list, axis=0).reset_index(drop=True)
wine_df.to_pickle('../pkl/00_wine_dot_com_data_{}.pkl'.format(str(url[0])))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
